In [40]:
import random

class BattleshipGame:
    def __init__(self, grid_size=5):
        self.grid_size = grid_size
        self.player_grid = [[' ' for _ in range(grid_size)] for _ in range(grid_size)]
        self.ai_grid = [[' ' for _ in range(grid_size)] for _ in range(grid_size)]

    def print_grid(self, grid):
        for row in grid:
            print(' | '.join(row))
            print('-' * (self.grid_size * 4 - 1))

    def place_ship(self, grid, ship):
        x, y = ship
        grid[x][y] = 'S'
        return grid

    def take_turn(self, grid, x, y):
        if grid[x][y] == 'S':
            grid[x][y] = 'X'  # Touché
            return True
        elif grid[x][y] == ' ':
            grid[x][y] = 'O'  # Manqué
            return False


In [41]:
class BattleshipGameAI(BattleshipGame):
    def __init__(self, grid_size=5, num_ships=3):
        super().__init__(grid_size)
        self.num_ships = num_ships
        self.ai_grid = self.random_place_ships(self.ai_grid)

    def random_place_ships(self, grid):
        ships_placed = 0
        while ships_placed < self.num_ships:
            x, y = random.randint(0, self.grid_size - 1), random.randint(0, self.grid_size - 1)
            if grid[x][y] == ' ':
                grid = self.place_ship(grid, (x, y))
                ships_placed += 1
        return grid

    def ai_take_turn(self):
        x, y = random.randint(0, self.grid_size - 1), random.randint(0, self.grid_size - 1)
        return self.take_turn(self.player_grid, x, y)

    
    def reset(self):
        # Réinitialiser la grille et placer les navires
        self.ai_grid = [[' ' for _ in range(self.grid_size)] for _ in range(self.grid_size)]
        self.ai_grid = self.random_place_ships(self.ai_grid)
        # Retourner l'état initial (peut-être une représentation de la grille)
        return self.grid_to_state(self.ai_grid)
        
    def step(self, action):
        # Appliquer une action (par exemple, un tir) et retourner le nouvel état, la récompense et si le jeu est terminé
        x, y = self.action_to_coords(action)
        hit = self.take_turn(self.ai_grid, x, y)
        reward = 1 if hit else -1
        done = self.check_game_over(self.ai_grid)  # Vérifie si le jeu est terminé
        next_state = self.grid_to_state(self.ai_grid)
        return next_state, reward, done, {}


    
    def grid_to_state(self, grid):
        # Convertir la grille en un état numérique pour l'apprentissage
        state = []
        for row in grid:
            for cell in row:
                if cell == ' ':
                    state.append(0)
                elif cell == 'S':
                    state.append(1)
                elif cell in ['X', 'O']:  # Marques pour les tirs réussis ou ratés
                    state.append(-1)
        return np.array([state])
    def action_to_coords(self, action):
        # Convertir une action en coordonnées de la grille
        return divmod(action, self.grid_size)
            
    def check_game_over(self, grid):
        # Vérifier si tous les navires ont été touchés
        for row in grid:
            if 'S' in row:  # S'il reste des navires non touchés
                return False
        return True  # Tous les navires ont été touchés

In [42]:
# Création du jeu
game = BattleshipGameAI()

# Affichage des grilles (pour la démonstration)
print("Grille du Joueur:")
game.print_grid(game.player_grid)
print("\nGrille de l'IA:")
game.print_grid(game.ai_grid)

# Placement des navires du joueur (à remplacer par les entrées utilisateur)
player_ship_coords = ["0,0", "1,2", "3,4"]
for coords in player_ship_coords:
    x, y = map(int, coords.split(','))
    game.player_grid = game.place_ship(game.player_grid, (x, y))

# Jouer un tour (exemple)
hit = game.ai_take_turn()
print("\nL'IA a attaqué :", hit)

# Afficher la grille du joueur après l'attaque de l'IA
print("\nGrille du Joueur après l'attaque de l'IA:")
game.print_grid(game.player_grid)


Grille du Joueur:
  |   |   |   |  
-------------------
  |   |   |   |  
-------------------
  |   |   |   |  
-------------------
  |   |   |   |  
-------------------
  |   |   |   |  
-------------------

Grille de l'IA:
S |   |   |   |  
-------------------
S |   |   |   |  
-------------------
  |   |   |   |  
-------------------
  | S |   |   |  
-------------------
  |   |   |   |  
-------------------

L'IA a attaqué : False

Grille du Joueur après l'attaque de l'IA:
S |   |   |   |  
-------------------
  |   | S |   |  
-------------------
  |   |   |   |  
-------------------
  |   |   |   | S
-------------------
  | O |   |   |  
-------------------


In [43]:
import numpy as np
import random
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam

class DQNAgent:
    def __init__(self, state_size, action_size):
        self.state_size = state_size
        self.action_size = action_size
        self.memory = []
        self.gamma = 0.95    # facteur de remise
        self.epsilon = 1.0   # exploration rate
        self.epsilon_min = 0.01
        self.epsilon_decay = 0.995
        self.learning_rate = 0.001
        self.model = self._build_model()

    def _build_model(self):
        # Construction du modèle de réseau de neurones pour Deep Q-learning
        model = Sequential()
        model.add(Dense(24, input_dim=self.state_size, activation='relu'))
        model.add(Dense(24, activation='relu'))
        model.add(Dense(self.action_size, activation='linear'))
        model.compile(loss='mse', optimizer=Adam(learning_rate=self.learning_rate))

        return model

    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))

    def act(self, state):
        if np.random.rand() <= self.epsilon:
            return random.randrange(self.action_size)
        act_values = self.model.predict(state)
        return np.argmax(act_values[0])

    def replay(self, batch_size):
        minibatch = random.sample(self.memory, batch_size)
        for state, action, reward, next_state, done in minibatch:
            target = reward
            if not done:
                target = reward + self.gamma * np.amax(self.model.predict(next_state)[0])
            target_f = self.model.predict(state)
            target_f[0][action] = target
            self.model.fit(state, target_f, epochs=1, verbose=1)
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay


In [46]:
def train_dqn_agent(agent, game, episodes, batch_size):
    for e in range(episodes):
        # Réinitialiser l'état au début de chaque jeu
        state = game.reset()  # Reset renvoie l'état initial du jeu
        state = np.reshape(state, [1, agent.state_size])
        total_reward = 0  # Récompense cumulée pour l'épisode

        for time in range(500):  # Nombre maximal de tours par jeu
            # L'agent choisit une action
            action = agent.act(state)

            # Appliquer l'action au jeu
            next_state, reward, done, _ = game.step(action)
            total_reward += reward  # Ajouter la récompense à la récompense cumulée
            next_state = np.reshape(next_state, [1, agent.state_size])

            # Mémoriser l'expérience précédente
            agent.remember(state, action, reward, next_state, done)

            # Passer à l'état suivant
            state = next_state

            if done:
                # Le jeu est terminé, sortir de la boucle
                break

            if len(agent.memory) > batch_size:
                # Entraîner l'agent avec un sous-ensemble de l'expérience de jeu
                agent.replay(batch_size)

        print(f"Épisode: {e+1}/{episodes}, Total des récompenses: {total_reward}")

    # Après la fin de l'entraînement, vous pouvez réduire l'epsilon à sa valeur minimale
    agent.epsilon = agent.epsilon_min


In [ ]:
# Paramètres
state_size = 25  # Par exemple, une grille de 5x5 aplaties en vecteur de taille 25
action_size = 25  # 25 actions possibles (tirer sur chaque case)
episodes = 1000
batch_size = 32

# Création de l'agent
agent = DQNAgent(state_size, action_size)

# Création du jeu
game = BattleshipGameAI()

# Entraînement
train_dqn_agent(agent, game, episodes, batch_size)


1/1 [==============================] - 0s 4ms/step - loss: 4.6278e-04
Épisode: 1/1000, Total des récompenses: -29
1/1 [==============================] - 0s 4ms/step - loss: 0.0017
Épisode: 2/1000, Total des récompenses: -6
1/1 [==============================] - 0s 8ms/step - loss: 8.2988e-05
Épisode: 3/1000, Total des récompenses: -10
1/1 [==============================] - ETA: 0s